In [1]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# DotProductComp

`DotProductComp` performs a dot product between two compatible inputs.  It may be vectorized to provide the result at one or more points simultaneously.

$$
    c_i = \bar{a}_i \cdot \bar{b}_i
$$

## DotProductComp Options

The default `vec_size` is 1, providing the dot product of $a$ and $b$ at a single
point.  The lengths of $a$ and $b$ are provided by option `length`.

Other options for DotProductComp allow the user to rename the input variables $a$ and $b$ and the output $c$, as well as specifying their units.

In [2]:
import openmdao.api as om
om.show_options_table("openmdao.components.dot_product_comp.DotProductComp")

Option,Default,Acceptable Values,Acceptable Types,Description
a_name,a,N/A,['str'],The variable name for input vector a.
a_units,N/A,N/A,['str'],The units for vector a.
always_opt,False,"[True, False]",['bool'],"If True, force nonlinear operations on this component to be included in the optimization loop even if this component is not relevant to the design variables and responses."
b_name,b,N/A,['str'],The variable name for input vector b.
b_units,N/A,N/A,['str'],The units for vector b.
c_name,c,N/A,['str'],The variable name for output vector c.
c_units,N/A,N/A,['str'],The units for vector c.
distributed,False,"[True, False]",['bool'],"If True, set all variables in this component as distributed across multiple processes"
length,3,N/A,['int'],The length of vectors a and b
run_root_only,False,"[True, False]",['bool'],"If True, call compute, compute_partials, linearize, apply_linear, apply_nonlinear, and compute_jacvec_product only on rank 0 and broadcast the results to the other ranks."


## DotProductComp Constructor

The call signature for the `DotProductComp` constructor is:

```{eval-rst}
    .. automethod:: openmdao.components.dot_product_comp.DotProductComp.__init__
        :noindex:
```

## DotProductComp Usage

There are often situations when numerous products need to be computed, essentially in parallel.
You can reduce the number of components required by having one `DotProductComp` perform multiple operations.
This is also convenient when the different operations have common inputs.

The ``add_product`` method is used to create additional products after instantiation.

```{eval-rst}
    .. automethod:: openmdao.components.dot_product_comp.DotProductComp.add_product
       :noindex:
```

## DotProductComp Example

In the following example DotProductComp is used to compute instantaneous power as the
dot product of force and velocity at 100 points simultaneously.  Note the use of
`a_name`, `b_name`, and `c_name` to assign names to the inputs and outputs.
Units are assigned using `a_units`, `b_units`, and `c_units`.
Note that no internal checks are performed to ensure that `c_units` are consistent
with `a_units` and `b_units`.


In [3]:
import numpy as np
import openmdao.api as om

n = 24

p = om.Problem()

dp_comp = om.DotProductComp(vec_size=n, length=3, a_name='F', b_name='v', c_name='P',
                            a_units='N', b_units='m/s', c_units='W')

p.model.add_subsystem(name='dot_prod_comp', subsys=dp_comp,
                     promotes_inputs=[('F', 'force'), ('v', 'vel')])

p.setup()

p.set_val('force', np.random.rand(n, 3))
p.set_val('vel', np.random.rand(n, 3))

p.run_model()

# Verify the results against numpy.dot in a for loop.
expected = []
for i in range(n):
    a_i = p.get_val('force')[i, :]
    b_i = p.get_val('vel')[i, :]
    expected.append(np.dot(a_i, b_i))

    actual_i = p.get_val('dot_prod_comp.P')[i]
    rel_error = np.abs(expected[i] - actual_i)/actual_i
    assert rel_error < 1e-9, f"Relative error: {rel_error}"

print(p.get_val('dot_prod_comp.P', units='kW'))

[0.00056672 0.0011848  0.0007009  0.00082938 0.00022927 0.00084332
 0.00111579 0.0014637  0.00045862 0.0011765  0.0002101  0.00033286
 0.00134953 0.00095854 0.00049979 0.00039304 0.00070145 0.00053748
 0.00087657 0.00068174 0.00130268 0.00081036 0.00097762 0.00108458]


In [4]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(p.get_val('dot_prod_comp.P', units='kW'), np.array(expected)/1000.)

1.0521795736255595e-16

## DotProductComp Example with Multiple Products

When defining multiple products:

- An input name in one call to `add_product` may not be an output name in another call, and vice-versa.
- The units and shape of variables used across multiple products must be the same in each one.

In [5]:
n = 24

p = om.Problem()

dp_comp = om.DotProductComp(vec_size=n, length=3,
                            a_name='F', b_name='d', c_name='W',
                            a_units='N', b_units='m', c_units='J')

dp_comp.add_product(vec_size=n, length=3,
                    a_name='F', b_name='v', c_name='P',
                    a_units='N', b_units='m/s', c_units='W')

p.model.add_subsystem(name='dot_prod_comp', subsys=dp_comp,
                      promotes_inputs=[('F', 'force'), ('d', 'disp'), ('v', 'vel')])

p.setup()

p.set_val('force', np.random.rand(n, 3))
p.set_val('disp', np.random.rand(n, 3))
p.set_val('vel', np.random.rand(n, 3))

p.run_model()

# Verify the results against numpy.dot in a for loop.
expected_P = []
expected_W = []
for i in range(n):
    a_i = p.get_val('force')[i, :]

    b_i = p.get_val('disp')[i, :]
    expected_W.append(np.dot(a_i, b_i))

    actual_i = p.get_val('dot_prod_comp.W')[i]
    rel_error = np.abs(actual_i - expected_W[i])/actual_i
    assert rel_error < 1e-9, f"Relative error: {rel_error}"

    b_i = p.get_val('vel')[i, :]
    expected_P.append(np.dot(a_i, b_i))

    actual_i = p.get_val('dot_prod_comp.P')[i]
    rel_error = np.abs(expected_P[i] - actual_i)/actual_i
    assert rel_error < 1e-9, f"Relative error: {rel_error}"

print(p.get_val('dot_prod_comp.W', units='kJ'))

[0.00137106 0.0008092  0.00049821 0.0004493  0.00077646 0.00046534
 0.0007641  0.00066497 0.00064934 0.00065087 0.00070043 0.0013951
 0.00029199 0.00053022 0.00077758 0.00074522 0.00071769 0.00112375
 0.00053925 0.001025   0.00112291 0.0012813  0.00041722 0.00118248]


In [6]:
print(p.get_val('dot_prod_comp.P', units='kW'))

[1.26965138e-03 7.74451483e-04 6.48237012e-04 4.28106931e-04
 9.81909920e-04 4.85303805e-04 9.04354842e-04 4.14975631e-04
 8.34744388e-04 9.97353451e-04 6.30827420e-04 9.26378367e-04
 8.12257147e-04 1.14919463e-03 2.84847263e-04 2.50190375e-04
 1.41094338e-03 9.96576422e-04 6.90663946e-04 6.86358933e-04
 1.17481382e-03 8.01523160e-04 4.50360747e-04 9.81779759e-05]


In [7]:
assert_near_equal(p.get_val('dot_prod_comp.W', units='kJ'), np.array(expected_W)/1000.)
assert_near_equal(p.get_val('dot_prod_comp.P', units='kW'), np.array(expected_P)/1000.)

1.0681215503556445e-16